In [1]:
import importlib

if importlib.util.find_spec('librosa') is None:
  ! pip install --user librosa
else:
  print('librosa already installed')

if importlib.util.find_spec('gradio') is None:
  ! pip install gradio
else:
  print('gradio already installed')

if importlib.util.find_spec('transformers') is None:
    print('Installing transformers...')
    ! pip install transformers
    ! pip install pyctcdecode
    ! pip install https://github.com/kpu/kenlm/archive/master.zip
else:
    print('libraries are already installed')

librosa already installed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. T

In [2]:
from pathlib import Path
# from scipy.io import wavfile
# import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
# import seaborn as sns
# import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
# import os
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Reshape
from tensorflow.keras.models import Sequential
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.regularizers import l1, l2
from transformers import AutoProcessor, TFWav2Vec2Model
# from tensorflow.keras import mixed_precision
from sklearn.metrics import classification_report
import json
import librosa
import gradio as gr

## Mount Drive

In [3]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [4]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
  # intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Mounted at /content/gdrive


## Read and define model

In [5]:
SAMPLE_RATE = 16000

In [6]:
model_cnn = Sequential([Conv2D(32, (2, 2), activation='relu', padding='same', input_shape=(32, 20, 1)),
                        MaxPooling2D((2, 2)),
                        Conv2D(32, (2, 2), activation='relu', padding='same'),
                        MaxPooling2D((2, 2)),
                        Conv2D(32, (2, 2), activation='relu', padding='same'),
                        MaxPooling2D((2, 2)),
                        Flatten(),
                        # Dropout(.2),
                        # Dense(32, activation='relu'),
                        Dropout(.175),
                        Dense(20, activation='softmax')])
model_cnn.load_weights(intermediate_folder / 'best_model_mfcc_cnn.ckp')
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 20, 32)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 10, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 10, 32)        4128      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 5, 32)          4128      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 2, 32)         0

In [7]:
model_name = 'facebook/wav2vec2-base-960h'
processor = AutoProcessor.from_pretrained(model_name)
model_wav2vec_hf = TFWav2Vec2Model.from_pretrained(model_name)

model_wav2vec_finetune = Sequential([
    Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
    MaxPooling2D((49, 1)),
    # Conv2D(2, (3, 1), activation='relu', padding='same'),
    # MaxPooling2D((3, 1)),
    # Conv2D(2, (3, 1), activation='relu', padding='same'),
    # MaxPooling2D((3, 1)),
    Flatten(),
    Dropout(.5),
    # Dense(32, activation='relu'),
    # Dropout(.5),
    # Dense(128, activation='relu'),
    # Dropout(.2),
    Dense(20, activation='softmax')])

input_ids = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='float64', name='input')
attention_masks = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='int32', name='mask')

output = model_wav2vec_hf([input_ids, attention_masks])
output = output[0]
output = model_wav2vec_finetune(output)

model_wav2vec_full = tf.keras.models.Model(inputs={"input": input_ids, "mask": attention_masks}, outputs=output)
model_wav2vec_full.load_weights(intermediate_folder / 'full_model_weights.h5')
model_wav2vec_full.summary()


TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2Model were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input[0][0]',                  
 Model)                         Output(last_hidden_               'mask[0][0]']                   
                                state=(None, 49, 76                                               
                                8),                                                           

## Read data

In [8]:
X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy')
# X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy').astype(float)
X_test.shape

(4689, 16000)

In [9]:
y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_test_labels.shape

(4689,)

In [10]:
with open(intermediate_folder / 'labels.json', 'r') as file:
    labels_dict = json.load(file)
labels_dict

{'down': 0,
 'eight': 1,
 'five': 2,
 'four': 3,
 'go': 4,
 'left': 5,
 'nine': 6,
 'no': 7,
 'off': 8,
 'on': 9,
 'one': 10,
 'right': 11,
 'seven': 12,
 'six': 13,
 'stop': 14,
 'three': 15,
 'two': 16,
 'up': 17,
 'yes': 18,
 'zero': 19}

In [11]:
# y_train = np.array([labels_dict[lbl] for lbl in y_train_labels])
# y_val = np.array([labels_dict[lbl] for lbl in y_val_labels])
y_test = np.array([labels_dict[lbl] for lbl in y_test_labels])

# del y_train_labels
# del y_val_labels
del y_test_labels

## Function of predict

In [12]:
MEAN = 33.06522526955586
STD = 142.73227249991973
MEAN, STD

(33.06522526955586, 142.73227249991973)

In [13]:
SAMPLE_RATE = 16000

In [14]:
labels_dict_reversed = {v:k for k, v in labels_dict.items()}
labels_dict_reversed

{0: 'down',
 1: 'eight',
 2: 'five',
 3: 'four',
 4: 'go',
 5: 'left',
 6: 'nine',
 7: 'no',
 8: 'off',
 9: 'on',
 10: 'one',
 11: 'right',
 12: 'seven',
 13: 'six',
 14: 'stop',
 15: 'three',
 16: 'two',
 17: 'up',
 18: 'yes',
 19: 'zero'}

In [15]:
def predict_with_expected(data, expected):
  data_transformed = np.expand_dims(np.expand_dims(librosa.feature.mfcc(y=data.astype(float), sr=SAMPLE_RATE).transpose(1, 0), axis=-1), axis=0)
  data_transformed = (data_transformed - MEAN) / STD
  y_pred = model_cnn.predict(data_transformed, verbose=0)
  print(f'Largest class: {y_pred.argmax(axis=1)[0]}, expected: {expected}, same: {y_pred.argmax(axis=1)[0] == expected}')

In [16]:
for ind in np.random.randint(len(X_test), size=10):
  predict_with_expected(X_test[ind], y_test[ind])

Largest class: 19, expected: 19, same: True
Largest class: 6, expected: 5, same: False
Largest class: 6, expected: 6, same: True
Largest class: 10, expected: 10, same: True
Largest class: 16, expected: 16, same: True
Largest class: 1, expected: 1, same: True
Largest class: 18, expected: 18, same: True
Largest class: 11, expected: 11, same: True
Largest class: 5, expected: 5, same: True
Largest class: 15, expected: 15, same: True


In [17]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [45]:
num_used = 0

In [46]:
def predict(option, data_mic, data_upload):
  global num_used
  num_used += 1
  print(f'Being used {num_used} time!')
  data = data_mic if data_mic else data_upload
  if data:
    sample_rate = data[0]
    audio = data[1]
    orig_audio = audio
  else:
    return "", None, None

#   print(f'---------------------')
  if sample_rate != SAMPLE_RATE:
    # print(f'Before resampling {sample_rate=}, {len(audio)=}')
    audio = librosa.resample(y=audio.astype(float), orig_sr=sample_rate, target_sr=SAMPLE_RATE)
  else:
    # print(f'Sample rate as expected {sample_rate=}, {len(audio)=}')
    audio = audio.astype(np.float32)

  # print(f'Before {SAMPLE_RATE=}, {len(audio)=}')
  if len(audio) > SAMPLE_RATE:
    audio, inds = librosa.effects.trim(audio)
    # print(f'After trimming {SAMPLE_RATE=}, {len(audio)=}, {inds=}')

  # if len(audio) > SAMPLE_RATE:
  #   audio = audio[:SAMPLE_RATE]
  #   print(f'After removing end {SAMPLE_RATE=}, {len(audio)=}')

  if len(audio) > SAMPLE_RATE:
    # print(f'Finding the loudest second')
    frames = librosa.util.frame(audio,
                                frame_length=SAMPLE_RATE,
                                hop_length=SAMPLE_RATE // 10,
                                axis=0
                                )
    # print(f'{len(frames)=}, {frames.shape=}')

    # Calculate the RMS (root mean square) of each frame
    rms = np.sqrt(np.mean(np.abs(frames)**2, axis=1))
    # print(f'{rms.shape=} {rms=}')

    # Find the index of the frame with the highest RMS
    loudest_frame_index = np.argmax(rms)
    # print(f'Found loudest frame {loudest_frame_index} from total {len(rms)}')

    # Get the audio for this frame
    audio = frames[loudest_frame_index, :]

  if len(audio) < SAMPLE_RATE:
    # print(f'Padding')
    audio = np.pad(audio, (0, SAMPLE_RATE - len(audio)))

  # print(f'After putting as 1 second {SAMPLE_RATE=}, {len(audio)=}')

  if option == 'CNN-based' or option is None:
    data_transformed = np.expand_dims(np.expand_dims(librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE).transpose(1, 0), axis=-1), axis=0)
    data_transformed = (data_transformed - MEAN) / STD
    # print(f'Shape on input to model {data_transformed.shape=}')
    y_pred = model_cnn.predict(data_transformed, verbose=0)
    # print(f'{y_pred.shape=}')
  else:
    data_transformed = processor(audio, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
    # print(data_transformed.shape)
    y_pred = model_wav2vec_full.predict([data_transformed, tf.ones([1, SAMPLE_RATE], dtype=tf.int32)], verbose=0)

  bar_data = pd.DataFrame({'Probability': softmax(y_pred[0]).round(2),
                           'Word': [labels_dict_reversed[ind] for ind in range(len(labels_dict_reversed))]}
                           ).sort_values(by='Probability', ascending=False)
  # print(bar_data)
  if audio.shape == orig_audio.shape and (audio == orig_audio).all():
    res_audio = None
  else:
    res_audio = (SAMPLE_RATE, audio)

  return labels_dict_reversed[y_pred.argmax(axis=1)[0]], bar_data.iloc[:5] , res_audio

## Gradio

In [ ]:
demo = gr.Interface(fn=predict,
                    inputs=[gr.components.Radio(['CNN-based', 'wav2vec-based'], value='CNN-based', label='Choose model type. If not chosen, CNN-based model will be used.', show_label=True),
                            # gr.components.Dropdown(['CNN-based', 'wav2vec-based'], value='CNN-based', label='Choose model type', show_label=True),
                            gr.components.Audio(source="microphone", autoplay=True, label='Record a few seconds of audio with a short (< 1 sec) clear word in the middle', show_label=True),
                            gr.components.Audio(source="upload", autoplay=True, label='Or upload an audio file with one of the words', show_label=True)],
                    outputs=[gr.components.Textbox(label='Result - most probable word:', show_label=True),
                             gr.components.BarPlot(x='Word', y='Probability', vertical=False, label='Probas', y_lim=[0, 1], width=300, show_label=True),
                             gr.components.Audio(label='Transformed audio predicted on (sampling change, trimmed, padded, loudest second chosen etc.)', show_label=True, autoplay=False)
                             ],
                    live=True,
                    allow_flagging='never',
                    title='Audio word classification',
                    description="""<h3>Record or upload a file with audio with one of the following words:</h3>
                    <table>
                      <tr>
                        <td valign='top'>
                          <h3>Numbers:</h3>
                          <p>zero, one, two, three, four, five, six, seven, eight, nine</p>
                        </td>
                        <td valign='top'>
                          <h3>Additional words:</h3>
                          <p>yes, no, down, up, left, right, go, stop, on, off</p>
                        </td>
                      </tr>
                    </table>
                    2 options for models are given: Simpler CNN-based model (14K params, accuracy 88%), HF wav2vec-based model (94M params, accuracy 95%).
                    <br>You can download audio file examples <a href="https://drive.google.com/drive/folders/1JhyPQ6Zl2FxwDMx1jCwpXhqt5jHzOr-t">from here</a>
                    <br>If the audio is both recorded and uploaded, the recorded audio is used.""")

demo.launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:272: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:272: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://41930e128098245519.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Being used 1 time!


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:183: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Being used 2 time!


...

/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:183: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Being used 34 time!
Being used 35 time!


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:183: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Being used 36 time!


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:183: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
